So yesterday, I learned that there is an asOf=<date> feature in the LB from **Ben Gorman**. So I thought it would be cool to use this to generate plots of the progress of the LB.

This kernel will scrape the leaderboard from the start of the competition until now and generate a line plot of the scores of the specified users. **Unfortunately, requests.get() does not seem to work from a Kaggle kernel** (if someone has a solution for this, please post it!). Therefore, you'll need to execute this code locally.

For the example image below, I just picked 4 random names!

![Example][1]


  [1]: http://i68.tinypic.com/2l8clfm.jpg

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import datetime
from lxml import html
import requests
import urllib

In [ ]:
PLAYER_NAMES = [
                'UGent Elves'
                ] # A line-plot will be generated for all players in this list
SAMPLE_HOURS = 12 # Indicates the interval in hours between the sampling
COMPETITION_START = datetime.datetime(year=2016, month=12, day=21)
URL = "https://www.kaggle.com/c/santas-uncertain-bags/leaderboard/public?asOf="

In [ ]:
import ctypes
libc = ctypes.cdll.LoadLibrary('libc.so.6')
res_init = libc.__res_init
# ...
res_init()
print(urllib.request.urlopen('http://yahoo.com').read())

In [ ]:
sample_time = datetime.timedelta(hours=SAMPLE_HOURS)
today = datetime.datetime.today()
current_time = COMPETITION_START
lb_scores = {}

for player in PLAYER_NAMES:
    lb_scores[player] = []

while current_time < today:
    # https://www.kaggle.com/c/santas-uncertain-bags/leaderboard/public?asOf=2017-1-1T12:00
    time_string = 'T'.join(str(current_time)[:-3].split())
    print(URL+time_string)
    page = requests.get(URL+time_string)
    tree = html.fromstring(page.content)
    current_time += sample_time
    for player in PLAYER_NAMES:
        found = False
        for row in tree.xpath('//tr[starts-with(@id, "team")]'):
            player_name = row.xpath('td/div/a/text()')[0].replace('\r', '').replace('\n', '').lstrip().rstrip()
            if player_name == player:
                player_score = row.xpath('td/abbr[@class="score"]/text()')
                if len(player_score):
                    lb_scores[player].append((time_string, float(player_score[0])))
                    found = True
        if not found:
            lb_scores[player].append((time_string, 0))

In [ ]:
plt.xkcd()

for player in lb_scores:
    plt.plot([x[1] for x in lb_scores[player]], label=player)

plt.xticks(range(len(lb_scores[PLAYER_NAMES[0]])), [x[0][5:] for x in lb_scores[PLAYER_NAMES[0]]], rotation='vertical')
plt.ylim([34000, 37000])
plt.legend()
plt.show()

![Example][1]


  [1]: http://i68.tinypic.com/2l8clfm.jpg